# Idébank

This app will be used to collect ideas from the SU Center of Digital Health team, store them on a [Google Sheet](https://docs.google.com/spreadsheets/d/1coKcUQWYzuO6gvKIjIpLO_6JbNz33kJpIP0daWIL2AI/)

In [1]:
!pip install pandas
!pip install google-api-python-client
!pip install protobuf-compiler
!pip install streamlit

  ERROR: Command errored out with exit status 1:
   command: 'c:\users\dsinz1\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\DSINZ1\\AppData\\Local\\Temp\\pip-install-r1jpwr2r\\grpcio_409b2f7d405b4e67b82446e0a9689159\\setup.py'"'"'; __file__='"'"'C:\\Users\\DSINZ1\\AppData\\Local\\Temp\\pip-install-r1jpwr2r\\grpcio_409b2f7d405b4e67b82446e0a9689159\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\DSINZ1\AppData\Local\Temp\pip-wheel-xu7j54g1'
       cwd: C:\Users\DSINZ1\AppData\Local\Temp\pip-install-r1jpwr2r\grpcio_409b2f7d405b4e67b82446e0a9689159\
  Complete output (319 lines):
  Found cython-generated files...
  running bdist_wheel
  running build
  running build_py
  running build_proj

    creating python_build\lib.win-amd64-3.8\grpc\_cython
    copying src\python\grpcio\grpc\_cython\__init__.py -> python_build\lib.win-amd64-3.8\grpc\_cython
    creating python_build\lib.win-amd64-3.8\grpc\framework\common
    copying src\python\grpcio\grpc\framework\common\cardinality.py -> python_build\lib.win-amd64-3.8\grpc\framework\common
    copying src\python\grpcio\grpc\framework\common\style.py -> python_build\lib.win-amd64-3.8\grpc\framework\common
    copying src\python\grpcio\grpc\framework\common\__init__.py -> python_build\lib.win-amd64-3.8\grpc\framework\common
    creating python_build\lib.win-amd64-3.8\grpc\framework\foundation
    copying src\python\grpcio\grpc\framework\foundation\abandonment.py -> python_build\lib.win-amd64-3.8\grpc\framework\foundation
    copying src\python\grpcio\grpc\framework\foundation\callable_util.py -> python_build\lib.win-amd64-3.8\grpc\framework\foundation
    copying src\python\grpcio\grpc\framework\foundation\future.py -> python_build


  Using cached protobuf_compiler-1.0.20-py3-none-any.whl (8.6 kB)
  Using cached colorama-0.3.3-py3-none-any.whl
  Using cached grpcio-1.18.0.tar.gz (14.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tqdm-4.31.1-py2.py3-none-any.whl (48 kB)
  Using cached bleach-2.1-py2.py3-none-any.whl (27 kB)
  Using cached grpcio_tools-1.18.0-cp38-cp38-win_amd64.whl
  Using cached termcolor-1.1.0-py3-none-any.whl
  Running setup.py clean for grpcio
Failed to build grpcio
    Running setup.py install for grpcio: started
    Running setup.py install for grpcio: finished with status 'error'


In [2]:
!pip install protobuf

In [3]:
import streamlit as st
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd

In [4]:
SCOPE = "https://www.googleapis.com/auth/spreadsheets"
SPREADSHEET_ID = "1coKcUQWYzuO6gvKIjIpLO_6JbNz33kJpIP0daWIL2AI"
SHEET_NAME = "Idebank"
GSHEET_URL = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}"

In [5]:
@st.experimental_singleton()
def connect_to_gsheet():
    # Create a connection object.
    credentials = service_account.Credentials.from_service_account_info(
        st.secrets["gcp_service_account"],
        scopes=[SCOPE],
    )

    service = build("sheets", "v4", credentials=credentials)
    gsheet_connector = service.spreadsheets()
    return gsheet_connector


def get_data(gsheet_connector) -> pd.DataFrame:
    values = (
        gsheet_connector.values()
        .get(
            spreadsheetId=SPREADSHEET_ID,
            range=f"{SHEET_NAME}!A:E",
        )
        .execute()
    )

    df = pd.DataFrame(values["values"])
    df.columns = df.iloc[0]
    df = df[1:]
    return df


def add_row_to_gsheet(gsheet_connector, row) -> None:
    values = (
        gsheet_connector.values()
        .append(
            spreadsheetId=SPREADSHEET_ID,
            range=f"{SHEET_NAME}!A:E",
            body=dict(values=row),
            valueInputOption="USER_ENTERED",
        )
        .execute()
    )


st.set_page_config(page_title="Idébank", page_icon="💡", layout="centered")

st.title("🗳️💡 Idébank")

gsheet_connector = connect_to_gsheet()

st.sidebar.write(
    f"Denna app ska användas för att samla in idéer från SU Center of Digital Hälsa-teamet, lagra dem på ett [Google Sheet]({GSHEET_URL})  sorterat efter kategori och prioritetsnivå."
)

st.sidebar.write(
    f"[Read more](https://docs.streamlit.io/knowledge-base/tutorials/databases/public-gsheet) about connecting your Streamlit app to Google Sheets."
)

form = st.form(key="annotation")

with form:
    cols = st.columns((1, 1))
    author = cols[0].text_input("Report author:")
    idea_category = cols[1].selectbox(
        "Idékategori:", ["Innovation", "Visualisering", "Finansiering", "Utbildning", "API", "Problelösning", "Utvekling", "Nätverka", "RPA", "IoT lab", "AR VR"], index=2
    )
    description = st.text_area("Beskrivning:")
    cols = st.columns(2)
    date = cols[0].date_input("Idéns inlämningsdatum:")
    idea_priority = cols[1].slider("Ide proritet:", 0, 1, 2, 3)
    submitted = st.form_submit_button(label="Skicka in")


if submitted:
    add_row_to_gsheet(
        gsheet_connector, [[author, idea_category, description, str(date), idea_priority]]
    )
    st.success("Jaaaj 🎉 Din idé har sparats!")
    st.balloons()

expander = st.expander("Se alla resultat")
with expander:
    st.write(f"Öppna originalet [Google Sheet]({GSHEET_URL})")
    st.dataframe(get_data(gsheet_connector))


2021-12-29 10:19:54.226 
  command:

    streamlit run C:\Users\DSINZ1\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\DSINZ1\\Downloads\\programmering\\idebank\\.streamlit\\secrets.toml'

In [ ]:
# to view this Streamlit app on a browser, run it with the following command:

#    streamlit run C:\Users\DSINZ1\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]